In [21]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt, hilbert

In [22]:
import numpy as np
from scipy.signal import iirfilter, filtfilt, hilbert, coherence, windows

# ---------- 1) Simulate two EMG-like signals ----------
fs = 1000
T  = 60.0
t  = np.arange(int(T*fs)) / fs

rng = np.random.default_rng(0)

# Broadband "EMG carrier" (20–150 Hz) as filtered white noise
def bandpass(x, fs, f1, f2, order=4):
    b, a = iirfilter(order, [f1, f2], btype='band', ftype='butter', fs=fs)
    return filtfilt(b, a, x)

x_car = bandpass(rng.standard_normal(t.size), fs, 20, 150)
y_car = bandpass(rng.standard_normal(t.size), fs, 20, 150)

# Add a weak, phase-locked 20 Hz component with a fixed lag (e.g., 30°)
f0 = 20
lag = np.deg2rad(30)
x_sig = 0.8*np.sin(2*np.pi*f0*t)                      # driver
y_sig = 0.8*np.sin(2*np.pi*f0*t + lag)                # lagged version

# Slow envelope (movement cycle) to create nonstationary amplitude
env = 1.0 + 0.8*np.maximum(0, np.sin(2*np.pi*0.25*t)) # rect-like, 0.25 Hz

# Raw EMG-like: envelope * (carrier + coherent sinusoid) + noise
x_raw = env*(x_car + x_sig) + 0.2*rng.standard_normal(t.size)
y_raw = env*(y_car + y_sig) + 0.2*rng.standard_normal(t.size)

# Rectify (common EMG preprocessing)
x_rect = np.abs(x_raw - np.mean(x_raw))
y_rect = np.abs(y_raw - np.mean(y_raw))

# ---------- 2) Demodulate via Hilbert (as in the paper) ----------
x_analytic = hilbert(x_rect)
y_analytic = hilbert(y_rect)

# Instantaneous phase; build amplitude-normalized "carrier-only" signals
phi_x = np.angle(x_analytic)
phi_y = np.angle(y_analytic)
x_demod = np.cos(phi_x)
y_demod = np.cos(phi_y)

# ---------- 3) Classical coherence (Welch) on demodulated signals ----------
win = windows.hamming(1024, sym=False)
noverlap = 768
f, Cxy_demod = coherence(x_demod, y_demod, fs=fs, window=win, nperseg=len(win), noverlap=noverlap)

# Peek at coherence around 20 Hz
idx20 = np.argmin(np.abs(f - 20))
print(f"Demodulated Welch coherence at ~20 Hz: {Cxy_demod[idx20]:.3f}")

# ---------- 4) Band-pass + Hilbert → PLV / wPLI in the 15–25 Hz band ----------
x_beta = bandpass(x_raw, fs, 15, 25)
y_beta = bandpass(y_raw, fs, 15, 25)

zx = hilbert(x_beta)          # analytic band signals
zy = hilbert(y_beta)

# Instantaneous phases in the band
phx = np.angle(zx)
phy = np.angle(zy)
dphi = phx - phy

# PLV (single time-averaged number)
plv = np.abs(np.mean(np.exp(1j*dphi)))
print(f"Band (15–25 Hz) PLV: {plv:.3f}")

# wPLI (time-averaged)
# Use imaginary part of cross-spectrum proxy: Im{zx * conj(zy)} = |zx||zy| sin(dphi)
im = np.imag(zx * np.conj(zy))
num = np.abs(np.mean(im))
den = np.mean(np.abs(im)) + 1e-12
wpli = num / den
print(f"Band (15–25 Hz) wPLI: {wpli:.3f}")

# (Optional) debiased wPLI (Vinck et al., 2011)
sum_im   = np.sum(im)
sum_im2  = np.sum(im**2)
sum_abs2 = np.sum(np.abs(im))**2
dwpli = (sum_im**2 - sum_im2) / (sum_abs2 - sum_im2 + 1e-12)
print(f"Band (15–25 Hz) debiased wPLI: {dwpli:.3f}")


Demodulated Welch coherence at ~20 Hz: 0.003
Band (15–25 Hz) PLV: 0.991
Band (15–25 Hz) wPLI: 1.000
Band (15–25 Hz) debiased wPLI: 1.000
